In [99]:
import requests
import pandas as pd
import duckdb
import json
import datamart_profiler
id="datamart.zenodo.3749559.e7fe094d-2f11-4814-ac1a-3326902e063c"
filename=id.replace(".","_").replace("-","_")+".parquet"
resp=requests.get('https://auctus.vida-nyu.org/api/v1/download/'+id)

In [100]:
data=resp.content.decode()
list=[x.split(',') for x in data.split('\n')]

df=pd.DataFrame(list[1:],columns=list[0])


In [101]:
profiles=datamart_profiler.process_dataset(df)
for c in profiles['columns']:
    print(c['name'],c['structural_type'])

/home/aromston/software/anaconda3/envs/spark/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/aromston/software/anaconda3/envs/spark/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/aromston/software/anaconda3/envs/spark/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/aromston/software/anaconda3/envs/spark/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value 

country http://schema.org/Text
date http://schema.org/Text
total_cases http://schema.org/Integer
new_cases http://schema.org/Integer
total_deaths http://schema.org/Integer
new_deaths http://schema.org/Integer
total_recovered http://schema.org/Integer
active_cases http://schema.org/Integer
servious_critical http://schema.org/Integer
total_cases_1M_pop http://schema.org/Float
total_deaths_1M_pop http://schema.org/Float
total_tests http://schema.org/Integer
 http://schema.org/Text


/home/aromston/software/anaconda3/envs/spark/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [68]:
df.to_parquet(filename)
df.to_csv('test_file.csv')

In [69]:
con=duckdb.connect("testdb.db")
res=con.sql(" describe select * from "+filename).fetchdf()
con.sql("select * from "+filename+" where 'Total cases'='81054.0'")

┌─────────┬─────────────┬───────────┬──────────────┬───┬────────────────┬──────────────┬─────────┬─────────┐
│ Country │ Total cases │ New cases │ Total deaths │ … │ Critical cases │ Cases/1M pop │   Lon   │   Lat
   │
│ varchar │   varchar   │  varchar  │   varchar    │   │    varchar     │   varchar    │ varchar │ varchar │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│                                                  0 rows                                                  │
└──────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [70]:
df_cpy=df.copy()
df_cpy
profiles=datamart_profiler.process_dataset(df_cpy)
for c in profiles['columns']:
    # print(c['name'],c['structural_type'])
    if "Integer" in c['structural_type']:
        df_cpy[c['name']]=df_cpy[c['name']].fillna(0).astype(float).astype(int)
    elif "Float" in c['structural_type']:
        df_cpy[c['name']]=df_cpy[c['name']].fillna(0).astype(float)
    elif "Text" in c['structural_type']:
        df_cpy[c['name']]=df_cpy[c['name']].fillna("null").astype(str)


/home/aromston/software/anaconda3/envs/spark/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/aromston/software/anaconda3/envs/spark/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/aromston/software/anaconda3/envs/spark/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/aromston/software/anaconda3/envs/spark/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value 

In [71]:
df_cpy.dtypes

Country             object
Total cases          int64
New cases           object
Total deaths        object
New deaths          object
Total recovered     object
Active cases         int64
Critical cases      object
Cases/1M pop       float64
Lon                float64
Lat\r               object
dtype: object

In [74]:
df_cpy.to_parquet("dummy_p.parquet")

In [85]:
con.sql(" describe select * from yellow.parquet ")

┌───────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│      column_name      │ column_type │  null   │   key   │ default │  extra  │
│        varchar        │   varchar   │ varchar │ varchar │ varchar │ varchar │
├───────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ VendorID              │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ tpep_pickup_datetime  │ TIMESTAMP   │ YES     │ NULL    │ NULL    │ NULL    │
│ tpep_dropoff_datetime │ TIMESTAMP   │ YES     │ NULL    │ NULL    │ NULL    │
│ passenger_count       │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ trip_distance         │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ RatecodeID            │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ store_and_fwd_flag    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ PULocationID          │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ DOLocationID          │ BIGINT      │ 

In [98]:
con.sql(" select * from dummy_p.parquet where \"Total cases\"<500")

┌──────────────────────┬─────────────┬───────────┬───┬──────────────┬───────────────────┬────────────────────┐
│       Country        │ Total cases │ New cases │ … │ Cases/1M pop │        Lon        │        Lat
         │
│       varchar        │    int64    │  varchar  │   │    double    │      double       │      varchar       │
├──────────────────────┼─────────────┼───────────┼───┼──────────────┼───────────────────┼────────────────────┤
│ Qatar                │         481 │ NaN       │ … │        167.0 │        51.2295295 │ 25.3336984
         │
│ Singapore            │         455 │ 23.0      │ … │         78.0 │       103.8194992 │ 1.357107
           │
│ Romania              │         433 │ 66.0      │ … │         23.0 │        24.6859225 │ 45.9852129
         │
│ Slovenia             │         414 │ 31.0      │ … │        199.0 │        14.4808369 │ 45.8133113
         │
│ India                │         396 │ 64.0      │ … │          0.3 │        78.6677428 │ 22.3511148
      